In [1]:
import pandas as pd
import numpy as np
import os
import yaml

In [2]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

In [8]:
pubs = pd.read_csv("_data/pubs.csv")
pubs['layout']="publication"
pubs['authors_list'] = pubs['authors']

for idx in range(len(pubs)):
    pubs['author_ids'].iloc[idx] = pubs['author_ids'].iloc[idx].replace(r', ',',').split(",")
    pubs['authors'].iloc[idx] = pubs['authors'].iloc[idx].replace(r', ',',').replace(u'\xa0', u' ').split(",")
    pubs['authors'].iloc[idx] = [item.lstrip() for item in pubs['authors'].iloc[idx]]
    pubs['title'].iloc[idx] = html_escape(pubs['title'].iloc[idx])
    if pubs['journal'].iloc[idx] is not np.NaN:
        pubs['journal'].iloc[idx] = html_escape(pubs['journal'].iloc[idx])

pubs.fillna("", inplace=True)

/var/folders/l_/3xydj4_d4gv5f314k5s9l4hw0000gs/T/ipykernel_65888/414038986.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  pubs['author_ids'].iloc[idx] = pubs['author_ids'].iloc[idx].replace(r', ',',').split(",")
/var/folders/l_/3xydj4_d

In [9]:
for row, item in pubs.iterrows():
    
    filename = '_publications/' +item['md_file_name']    
    md = "---\ntitle: \""   + html_escape(item.title) + '"\n'
    
    md += "\ndate: " + str(item.date) 

    md += "\nauthors_string: " + item.authors_list

    md += "\nauthors:"

    for author in item['authors']:
        md+= "\n   - "+author
    
    md += "\nauthor_ids:"

    for author_id in item['author_ids']:
        md += "\n   - "+author_id

    md += "\njournal: '" + html_escape(item.journal) + "'"

    md += "\nvolume: "+str(item.volume) 

    md += "\nissue: "+str(item.issue)

    md += "\npages: "+str(item.pages)

    md += "\nbook_title: '"+ item.book_title + "'"

    md += "\npublisher: '" + item.publisher + "'"

    if item.isbn != "":
        md += "\nisbn: " + str(int(item.isbn))
    else: 
        md+= "\nisbn: " 

    md += "\nabstract: '" + html_escape(item.abstract) + "'"

    md += "\nproject_id: " + item.project_id 

    md += "\npaper_url: " + item.paper_url 

    md += "\ndoi: " + item.doi

    md += "\ndata_loc: '"+ item.data_loc + "'"

    md += "\ncode_loc: '" + item.code_loc + "'"
    
    md += "\nfile: '/assets/publications/"+ item.file_name + "'"

    md += "\nfile_name: '"+item.file_name + "'" 

    md += "\ntype: " + item.type 

    #pub_str = item.authors_list+'. "'+ "<a href='{{publication.url}}'>"+ item.title+'</a>" '
    pub_str=""
    if item.type=="journal_article":

        pub_str += item.journal+ " ("+item.date[0:4]+") "
        if item.volume != "": 
            pub_str += str(int(item.volume))
        if item.issue != "":
            pub_str += " ("+str(int(item.issue))+")"
        if item.pages!= "": 
            if isinstance(item.pages, int):
                pub_str += str(int(item.pages))
            else: 
                pub_str += str(item.pages)

    elif item.type =="book_chapter": 
        pub_str += "In: " + item.book_title + " ("+item.date[0:4]+")"
    else: 
        pub_str += "("+item.date[0:4]+")"
    
    md += "\npub_str: '"+ pub_str + "'"

    md += "\nlayout: publication \n---"
       
    with open(filename, 'w') as f:
        f.write(md)